# Problema de designação de pessoas **I** para instrumentos **J** e vozes **L** no dia **K**
---

##  i - Conjunto de pessoas
##  J - Conjunto de instrumentos
##  k - Conjunto de dias
##  l - Conjunto de vozes

---

* Mij = Matriz Binária - Pessoa I toca Instrumento J

* Sil = Matriz Binária - Pessoa I toca canta na posição L

* PMj = Peso de cada instrumento J
* PSl = Peso de cada voz L

* Dk = Peso de cada dia K

### Solução
* Xijk - Matriz Binária no R3
* Yilk - Matriz Binária no R3
* Zik -  Matriz binária no R2

* **Se Xijk = 1 => A pessoa "i" tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 1 => A pessoa "i" cantará na posição 'l' no dia 'k'

* **Se Xijk = 0 => A pessoa "i" não tocará o instrumento 'j' no dia 'k'**

* Se Yijl = 0 => A pessoa "i" não cantará na posição 'l' no dia 'k'

* **Somatório em i de Xijk <= 1 -  Um instrumento 'j' só pode ser tocado por uma pessoa 'i' em um dia 'k'**

* Somatório em i de Yilk <= 1 - Uma voz 'l' só pode ser cantada por uma pessoa 'i' em um dia 'k'

* **Somatório em 'j' de Xijk <= 1 - A pessoa "i" tocará somente um instrumento instrumento 'j' no dia 'k'**

* Somatório em 'l' de Yilk <= 1 - A pessoa "i" cantará somente em uma posição "l" no dia "k"

* **Zik - Pessoa i participa no dia k**
* **Zik = 0 se Xijk e yilk = 0**
* **Zik = 1 se Xijk ou yilk = 1**


* **Zik + Zi(k+1) + Zi(k+2) + Zi(k+4) <= 2 para todo i - A pessoa "i" não participará mais do que dois dias seguidos.**


* **Xijk = 0 se PMij = 0**
* Yilk = 0 se PSil = 0


* Maximize Xijk * Mij * PMj * Dk + Yilk * Sil * PSl * Dk

In [161]:
import pandas as pd

file_name = 'Dados Louvor Shalom.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Vozes', 'Peso das Vozes', 'Peso dos Dias','Disponibilidade', 'Dias por Mês'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')
V = sheets_dict['Vozes'].set_index('Pessoas')

L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
LV = sheets_dict['Peso das Vozes'].set_index('Vozes').squeeze()
D = sheets_dict['Peso dos Dias'].set_index('Dias')

Disp = sheets_dict['Disponibilidade'].set_index('Dias')
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')

# Convertendo os índices para o tipo datetime
D.index = pd.to_datetime(D.index)
Disp.index = pd.to_datetime(Disp.index)

# Formatando os índices para o formato desejado
D.index = D.index.strftime('%d_%m_%Y')
Disp.index = Disp.index.strftime('%d_%m_%Y')

In [162]:
from pulp import *

days = 2

# Definir o problema
prob = LpProblem("Alocacao_de_Musicos", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (P.index, P.columns, D.index), cat='Binary')

# Função objetivo
prob += lpSum([X[i][j][k] * P.loc[i, j] * L[j] * D['Peso'][k] for i in P.index for j in P.columns for k in D.index])

# Restrições
for j in P.columns:
    for k in D.index:
        prob += lpSum([X[i][j][k] for i in P.index]) <= 1

for i in P.index:
    for k in D.index:
        prob += lpSum([X[i][j][k] for j in P.columns]) <= 1

for i in P.index:
    for k in range(len(D.index) - 3):
        prob += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+2]] for j in P.columns]) <= DPM['Quantidade'][i]

for i in P.index:
    for j in P.columns:
        for k in D.index:
            if P.loc[i, j] == 0:
                prob += X[i][j][k] == 0

# Resolver o problema
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/hmbarros/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/85a49e3bd6ef48d5accec8fc1485f7cf-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/85a49e3bd6ef48d5accec8fc1485f7cf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1355 COLUMNS
At line 9906 RHS
At line 11257 BOUNDS
At line 12428 ENDATA
Problem MODEL has 1350 rows, 1170 columns and 5911 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1326 - 0.00 seconds
Cgl0002I 871 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 100 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 69 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 43 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 32 strengthened rows, 0 substitution

1

In [163]:
# Imprimir o status da solução
print("Status:", LpStatus[prob.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob.objective))

# Dicionário para armazenar os resultados
resultados = {}

# Iterar sobre cada variável de decisão
for v in prob.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o instrumento e o dia
        _, pessoa, instrumento, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados:
            resultados[dia] = {}

        # Adicione a pessoa e o instrumento ao dia correspondente
        resultados[dia][instrumento] = pessoa

df = pd.DataFrame(columns=P.columns, index=D.index)


# Preencher o DataFrame com os resultados
for dia in resultados:
    for instrumento in resultados[dia]:
        df.loc[dia, instrumento] = resultados[dia][instrumento]

# # Substituir NaN por ''
df = df.fillna('')

# Convertendo os índices para o tipo datetime
df.index = pd.to_datetime(df.index, format='%d_%m_%Y')

# Formatando os índices para o formato desejado e criando um novo índice
formatted_index = df.index.strftime('%d/%m/%Y')

# Atribuindo o novo índice formatado ao DataFrame
df.index = formatted_index
df

Status: Optimal
Valor da função objetivo = 1326.0


,Bateria,Baixo,Teclado,Violão,Guitarra
Dias,,,,,
14/01/2024,Lucas,Daniel,Bernardo,Ricardo,Osvaldo
21/01/2024,Henrique,Ricardo,Mauri,Daniel,Osvaldo
28/01/2024,Marcela,Gabriel,Vinicius,Henrique,Osvaldo
04/02/2024,Lucas,Rafael,Bernardo,Osvaldo,Enzo
11/02/2024,Henrique,Ricardo,Vinicius,Osvaldo,Daniel
18/02/2024,Marcela,Gabriel,Henrique,Ricardo,Osvaldo
25/02/2024,Lucas,Enzo,Vinicius,Rafael,Osvaldo
03/03/2024,Henrique,Daniel,Vinicius,Ricardo,Osvaldo
10/03/2024,Marcela,Gabriel,Bernardo,Ricardo,Osvaldo


In [164]:
from pulp import *

days = 2

# Definir o problema
prob_v = LpProblem("Alocacao_de_Musicos", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (V.index, V.columns, D.index), cat='Binary')

# Função objetivo
prob_v += lpSum([X[i][j][k] * V.loc[i, j] * LV[j] * D['Peso'][k] for i in V.index for j in V.columns for k in D.index])

# Restrições
for j in V.columns:
    for k in D.index:
        prob_v += lpSum([X[i][j][k] for i in V.index]) <= 1

for i in V.index:
    for k in D.index:
        prob_v += lpSum([X[i][j][k] for j in V.columns]) <= 1

for i in V.index:
    for k in range(len(D.index) - 3):
        prob_v += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+2]] for j in V.columns]) <= DPM['Quantidade'][i]

for i in V.index:
    for j in V.columns:
        for k in D.index:
            if V.loc[i, j] == 0:
                prob_v += X[i][j][k] == 0

# Resolver o problema
prob_v.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/hmbarros/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/93028703a9b34e97bc110eb5a4b6d412-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/93028703a9b34e97bc110eb5a4b6d412-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 978 COLUMNS
At line 7819 RHS
At line 8793 BOUNDS
At line 9730 ENDATA
Problem MODEL has 973 rows, 936 columns and 4539 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 975 - 0.01 seconds
Cgl0002I 507 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 77 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 70 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 60 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 51 strengthened rows, 0 substitutions
Cgl00

1

In [167]:
# Imprimir o status da solução
print("Status:", LpStatus[prob_v.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob_v.objective))

# Dicionário para armazenar os resultados_v
resultados_v = {}

# Iterar sobre cada variável de decisão
for v in prob_v.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o instrumento e o dia
        _, pessoa, instrumento, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados_v:
            resultados_v[dia] = {}

        # Adicione a pessoa e o instrumento ao dia correspondente
        resultados_v[dia][instrumento] = pessoa

df_v = pd.DataFrame(columns=V.columns, index=D.index)


# Preencher o DataFrame com os resultados_v
for dia in resultados_v:
    for instrumento in resultados_v[dia]:
        df_v.loc[dia, instrumento] = resultados_v[dia][instrumento]

# # Substituir NaN por ''
df_v = df_v.fillna('')

# # Convertendo os índices para o tipo datetime
df_v.index = pd.to_datetime(df_v.index, format='%d_%m_%Y')

# Formatando os índices para o formato desejado e criando um novo índice
df_v.index =  df_v.index.strftime('%d/%m/%Y')

df_v

Status: Optimal
Valor da função objetivo = 975.0


,Ministração,Soprano,Tenor,Contralto
Dias,,,,
14/01/2024,Ricardo,Dani,Ariane,Henrique
21/01/2024,Ricardo,Everlyn,Dani,Júlia
28/01/2024,Everlyn,Henrique,Tassi,Marcela
04/02/2024,Dani,Ricardo,Ariane,Henrique
11/02/2024,Everlyn,Dani,Rafael,Ricardo
18/02/2024,Henrique,Marcela,Tassi,Everlyn
25/02/2024,Dani,Henrique,Ricardo,Ariane
03/03/2024,Ricardo,Rafael,Everlyn,Dani
10/03/2024,Henrique,Marcela,Everlyn,Tassi
